# Polar coordinates with jaxfun

In [35]:
from jaxfun import *
from jaxfun.Basespace import Domain
import numpy as np
import sympy as sp

#system = 'polar'
system = 'cylindrical'
r, theta, z = sp.symbols('r,theta,z', real=True)
    
if system == 'polar':
    C = get_CoordSys('C', sp.Lambda((r, theta), (r*sp.cos(theta), r*sp.sin(theta))))
    R = Composite(Legendre.Legendre, 20, bcs={'left': {'D': 0}, 'right': {'D': 0}}, domain=Domain(0.5, 1), name='R', fun_str='phi')
    T = Composite(Legendre.Legendre, 20, bcs={'left': {'D': 0}, 'right': {'D': 0}}, domain=Domain(0, np.pi), name='T', fun_str='psi')
    P = TensorProductSpace((R, T), coordinates=C, name='P')
    x, y = C.x, C.y
elif system == 'cylindrical':
    C = get_CoordSys('C', sp.Lambda((r, theta, z), (r*sp.cos(theta), r*sp.sin(theta), z)))
    R = Composite(Legendre.Legendre, 20, bcs={'left': {'D': 0}, 'right': {'D': 0}}, domain=Domain(0.5, 1), name='R', fun_str='phi')
    T = Composite(Legendre.Legendre, 20, bcs={'left': {'D': 0}, 'right': {'D': 0}}, domain=Domain(0, np.pi), name='T', fun_str='psi')
    Z = Legendre.Legendre(20, domain=Domain(0, 1), name='Z', fun_str='L')
    P = TensorProductSpace((R, T, Z), coordinates=C, name='P')
    x, y, z = C.x, C.y, C.z

In [36]:
u = TrialFunction(P, name='u')
du = Div(Grad(u))
du

Div(Grad(u(x, y, z; P)))

In [37]:
du.doit()

phi(r)*psi(theta)*Derivative(L(z), (z, 2)) + (r*psi(theta)*L(z)*Derivative(phi(r), (r, 2)) + psi(theta)*L(z)*Derivative(phi(r), r))/r + phi(r)*L(z)*Derivative(psi(theta), (theta, 2))/r**2

In [38]:
from sympy import srepr
srepr(du)

"Div(Grad(TrialFunction(x, y, z, Symbol('P'))))"

In [39]:
du = (x+y+z)*Div(Grad(u))
du

(x + y + z)*Div(Grad(u(x, y, z; P)))

In [40]:
srepr(du)

"Mul(Add(x, y, z), Div(Grad(TrialFunction(x, y, z, Symbol('P')))))"

In [41]:
du.doit()

(r*sin(theta) + r*cos(theta) + z)*(phi(r)*psi(theta)*Derivative(L(z), (z, 2)) + (r*psi(theta)*L(z)*Derivative(phi(r), (r, 2)) + psi(theta)*L(z)*Derivative(phi(r), r))/r + phi(r)*L(z)*Derivative(psi(theta), (theta, 2))/r**2)

In [42]:
Grad(u).doit()

(psi(theta)*L(z)*Derivative(phi(r), r))*C.b_r + (phi(r)*L(z)*Derivative(psi(theta), theta)/r**2)*C.b_theta + (phi(r)*psi(theta)*Derivative(L(z), z))*C.b_z

In [43]:
g = arguments.ScalarFunction('g', C)
g

g(x, y, z)

In [45]:
G = g.doit()
G

G(r, theta, z)

In [46]:
Div(Grad(g))

Div(Grad(g(x, y, z)))

In [47]:
Div(Grad(g)).doit()

Derivative(G(r, theta, z), (z, 2)) + (r*Derivative(G(r, theta, z), (r, 2)) + Derivative(G(r, theta, z), r))/r + Derivative(G(r, theta, z), (theta, 2))/r**2

In [48]:
P.tensorname

'R⊗T⊗Z'

In [49]:
V = VectorTensorProductSpace(P, name='V')
V.tensorname

'P×P×P'

In [50]:
V.name

'V'

In [51]:
v = TestFunction(V, name='v')
v

v(x, y, z; V)

In [52]:
v.doit()

(phi(r)*psi(theta)*L(z))*C.b_r + (phi(r)*psi(theta)*L(z))*C.b_theta + (phi(r)*psi(theta)*L(z))*C.b_z

In [53]:
Cross(v, v).doit()

0

In [54]:
c = Cross(v, C.b_r)
c

-Cross(C.b_r, v(x, y, z; V))

In [55]:
c.doit()

(-r*phi(r)*psi(theta)*L(z))*C.b_z + (phi(r)*psi(theta)*L(z)/r)*C.b_theta

In [56]:
d = Curl(v)
d

Curl(v(x, y, z; V))

In [57]:
d.doit()

((-r**2*phi(r)*psi(theta)*Derivative(L(z), z) + phi(r)*L(z)*Derivative(psi(theta), theta))/r)*C.b_r + ((phi(r)*psi(theta)*Derivative(L(z), z) - psi(theta)*L(z)*Derivative(phi(r), r))/r)*C.b_theta + ((r**2*psi(theta)*L(z)*Derivative(phi(r), r) + 2*r*phi(r)*psi(theta)*L(z) - phi(r)*L(z)*Derivative(psi(theta), theta))/r)*C.b_z

In [58]:
h = arguments.VectorFunction('h', C)
h

h(x, y, z)

In [59]:
H = h.doit()
H

(H_r(r, theta, z))*C.b_r + (H_\theta(r, theta, z))*C.b_theta + (H_z(r, theta, z))*C.b_z

In [60]:
D = d.doit()
D

((-r**2*phi(r)*psi(theta)*Derivative(L(z), z) + phi(r)*L(z)*Derivative(psi(theta), theta))/r)*C.b_r + ((phi(r)*psi(theta)*Derivative(L(z), z) - psi(theta)*L(z)*Derivative(phi(r), r))/r)*C.b_theta + ((r**2*psi(theta)*L(z)*Derivative(phi(r), r) + 2*r*phi(r)*psi(theta)*L(z) - phi(r)*L(z)*Derivative(psi(theta), theta))/r)*C.b_z

In [61]:
R = C.to_cartesian_vector(D)

In [62]:
R

(-(phi(r)*psi(theta)*Derivative(L(z), z) - psi(theta)*L(z)*Derivative(phi(r), r))*sin(theta) + (-r**2*phi(r)*psi(theta)*Derivative(L(z), z) + phi(r)*L(z)*Derivative(psi(theta), theta))*cos(theta)/r)*N.i + ((phi(r)*psi(theta)*Derivative(L(z), z) - psi(theta)*L(z)*Derivative(phi(r), r))*cos(theta) + (-r**2*phi(r)*psi(theta)*Derivative(L(z), z) + phi(r)*L(z)*Derivative(psi(theta), theta))*sin(theta)/r)*N.j + ((r**2*psi(theta)*L(z)*Derivative(phi(r), r) + 2*r*phi(r)*psi(theta)*L(z) - phi(r)*L(z)*Derivative(psi(theta), theta))/r)*N.k